# Gold Layer Creation

This notebook creates the gold layer with analytics-ready, aggregated data for business intelligence.

In [29]:
# Import required libraries
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Add project root to path
project_root = Path().absolute()
sys.path.insert(0, str(project_root))

from utils.database import DuckDBManager
from config.settings import config

# Initialize components
config.setup_logging()
db_manager = DuckDBManager()

print("🥇 Gold Layer Creation Started")
print("📊 Creating analytics-ready aggregated data...")

🥇 Gold Layer Creation Started
📊 Creating analytics-ready aggregated data...


## 1. Create Gold Schema

In [30]:
# Create gold schema
print("🏗️ Creating gold schema...")

try:
    db_manager.execute_sql("CREATE SCHEMA IF NOT EXISTS gold;")
    print("✅ Gold schema created")
    
    # Check if silver data exists
    silver_check = db_manager.execute_query("SELECT COUNT(*) as count FROM silver.cleaned_bond_data")
    silver_count = silver_check['count'].iloc[0]
    print(f"📊 Silver layer contains {silver_count:,} rows to aggregate")
    
except Exception as e:
    print(f"❌ Schema creation failed: {e}")

2025-06-18 17:34:10,568 - utils.database - INFO - S3 credentials configured
2025-06-18 17:34:10,569 - utils.database - INFO - DuckDB connection configured successfully
2025-06-18 17:34:10,572 - utils.database - INFO - SQL statement executed successfully
2025-06-18 17:34:10,575 - utils.database - INFO - Query executed successfully. Returned 1 rows


🏗️ Creating gold schema...
✅ Gold schema created
📊 Silver layer contains 0 rows to aggregate


## 2. Create Daily Aggregations

In [31]:
# Create daily aggregated data
print("📈 Creating daily aggregations...")

daily_aggregation_sql = """
CREATE OR REPLACE TABLE gold.daily_bond_metrics AS
SELECT 
    trade_date,
    
    -- Price metrics
    COUNT(*) as total_trades,
    AVG(price) as avg_price,
    MIN(price) as min_price,
    MAX(price) as max_price,
    STDDEV(price) as price_volatility,
    
    -- Yield metrics  
    AVG(yield_value) as avg_yield,
    MIN(yield_value) as min_yield,
    MAX(yield_value) as max_yield,
    STDDEV(yield_value) as yield_volatility,
    
    -- Data quality metrics
    SUM(CASE WHEN data_quality_flag = 'VALID' THEN 1 ELSE 0 END) as valid_trades,
    SUM(CASE WHEN data_quality_flag = 'INCOMPLETE' THEN 1 ELSE 0 END) as incomplete_trades,
    
    -- Calculated metrics
    ROUND(SUM(CASE WHEN data_quality_flag = 'VALID' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as data_quality_pct,
    
    current_timestamp as gold_created_timestamp
FROM silver.cleaned_bond_data
WHERE data_quality_flag IN ('VALID', 'INCOMPLETE')
GROUP BY trade_date
ORDER BY trade_date
"""

try:
    db_manager.execute_sql(daily_aggregation_sql)
    
    # Get row count
    count_result = db_manager.execute_query("SELECT COUNT(*) as row_count FROM gold.daily_bond_metrics")
    row_count = count_result['row_count'].iloc[0]
    print(f"✅ Daily metrics created for {row_count:,} trading days")
    
except Exception as e:
    print(f"❌ Daily aggregation failed: {e}")

2025-06-18 17:34:10,616 - utils.database - INFO - SQL statement executed successfully
2025-06-18 17:34:10,619 - utils.database - INFO - Query executed successfully. Returned 1 rows


📈 Creating daily aggregations...
✅ Daily metrics created for 0 trading days


## 3. Create Summary Views

In [32]:
# Create analytical views
print("📊 Creating analytical views...")

views_sql = """
-- Monthly summary view
CREATE OR REPLACE VIEW gold.monthly_bond_summary AS
SELECT 
    DATE_TRUNC('month', trade_date) as trade_month,
    SUM(total_trades) as monthly_total_trades,
    AVG(avg_price) as monthly_avg_price,
    AVG(avg_yield) as monthly_avg_yield,
    AVG(data_quality_pct) as monthly_avg_quality_pct,
    COUNT(*) as trading_days_in_month
FROM gold.daily_bond_metrics
GROUP BY DATE_TRUNC('month', trade_date)
ORDER BY trade_month;

-- Data quality dashboard view
CREATE OR REPLACE VIEW gold.data_quality_dashboard AS
SELECT 
    'Last 30 Days' as period,
    COUNT(*) as trading_days,
    SUM(total_trades) as total_trades,
    SUM(valid_trades) as valid_trades,
    SUM(incomplete_trades) as incomplete_trades,
    ROUND(AVG(data_quality_pct), 2) as avg_quality_pct,
    MIN(trade_date) as period_start,
    MAX(trade_date) as period_end
FROM gold.daily_bond_metrics
WHERE trade_date >= CURRENT_DATE - INTERVAL '30 days';
"""

try:
    db_manager.execute_sql(views_sql)
    print("✅ Analytical views created successfully")
    
    # Test the views
    monthly_summary = db_manager.execute_query("SELECT * FROM gold.monthly_bond_summary LIMIT 5")
    print("\n📊 Sample Monthly Summary:")
    print(monthly_summary)
    
except Exception as e:
    print(f"❌ Views creation failed: {e}")

2025-06-18 17:34:10,660 - utils.database - INFO - SQL statement executed successfully
2025-06-18 17:34:10,665 - utils.database - INFO - Query executed successfully. Returned 0 rows


📊 Creating analytical views...
✅ Analytical views created successfully

📊 Sample Monthly Summary:
Empty DataFrame
Columns: [trade_month, monthly_total_trades, monthly_avg_price, monthly_avg_yield, monthly_avg_quality_pct, trading_days_in_month]
Index: []


## 4. Performance Metrics and Summary

In [33]:
# Generate comprehensive summary
print("📋 Gold Layer Summary")
print("=" * 40)

try:
    # Get layer statistics
    layer_stats = {
        "Bronze Layer Records": "SELECT COUNT(*) as count FROM bronze.bond_data",
        "Silver Layer Records": "SELECT COUNT(*) as count FROM silver.cleaned_bond_data",
        "Gold Daily Metrics": "SELECT COUNT(*) as count FROM gold.daily_bond_metrics",
        "Gold Monthly Summaries": "SELECT COUNT(*) as count FROM gold.monthly_bond_summary"
    }
    
    print("📊 Data Lake Layer Statistics:")
    for stat_name, query in layer_stats.items():
        try:
            result = db_manager.execute_query(query)
            count = result['count'].iloc[0]
            print(f"  {stat_name}: {count:,}")
        except:
            print(f"  {stat_name}: Not available")
    
    # Get data quality overview
    quality_overview = db_manager.execute_query("SELECT * FROM gold.data_quality_dashboard")
    print("\n📈 Data Quality Overview (Last 30 Days):")
    print(quality_overview)
    
    # List all created objects
    print("\n🏗️ Created Gold Layer Objects:")
    print("  Tables:")
    print("    - gold.daily_bond_metrics")
    print("  Views:")
    print("    - gold.monthly_bond_summary")
    print("    - gold.data_quality_dashboard")
    
    print("\n🎉 Gold layer creation completed successfully!")
    print("📌 Your data lake is now ready for analytics and BI tools!")
    
except Exception as e:
    print(f"❌ Summary generation failed: {e}")

2025-06-18 17:34:10,701 - utils.database - INFO - Query executed successfully. Returned 1 rows
2025-06-18 17:34:10,703 - utils.database - INFO - Query executed successfully. Returned 1 rows
2025-06-18 17:34:10,706 - utils.database - INFO - Query executed successfully. Returned 1 rows
2025-06-18 17:34:10,708 - utils.database - INFO - Query executed successfully. Returned 1 rows


📋 Gold Layer Summary
📊 Data Lake Layer Statistics:
  Bronze Layer Records: 100,822,111
  Silver Layer Records: 0
  Gold Daily Metrics: 0
  Gold Monthly Summaries: 0


2025-06-18 17:34:10,715 - utils.database - INFO - Query executed successfully. Returned 1 rows



📈 Data Quality Overview (Last 30 Days):
         period  trading_days  total_trades  valid_trades  incomplete_trades  \
0  Last 30 Days             0           NaN           NaN                NaN   

   avg_quality_pct period_start period_end  
0              NaN          NaT        NaT  

🏗️ Created Gold Layer Objects:
  Tables:
    - gold.daily_bond_metrics
  Views:
    - gold.monthly_bond_summary
    - gold.data_quality_dashboard

🎉 Gold layer creation completed successfully!
📌 Your data lake is now ready for analytics and BI tools!
